In [10]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdMolTransforms import ComputeCentroid
import numpy as np
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform, cdist
# This script provides functions to read/write XYZ files, extract connectivity,
# select anchor atoms, sample directions on a unit sphere, and attach fragments
import argparse
COVALENT_RADII = {
    'H': 0.31, 'C': 0.76, 'N': 0.71, 'O': 0.66,
    'F': 0.57, 'P': 1.07, 'S': 1.05, 'Cl':1.02,
    'Br':1.20, 'I':1.39
}

def extract_connectivity(xyz_df, tol=0.45):
    coords = np.vstack([xyz_df.x, xyz_df.y, xyz_df.z]).T
    symbols = xyz_df.atom.values
    dmat = squareform(pdist(coords))
    idx_i, idx_j = np.triu_indices(len(symbols), k=1)
    df = pd.DataFrame({'i': idx_i, 'j': idx_j, 'd': dmat[idx_i, idx_j]})
    df['sym_i'] = symbols[df.i]
    df['sym_j'] = symbols[df.j]
    to_drop = []
    for k, r in df.iterrows():
        a, b, d = r.sym_i, r.sym_j, r.d
        # H rules
        if d == 0 or (a=='H' and b not in {'O','N','F'}) or (b=='H' and a not in {'O','N','F'}) or (a=='H' and b=='H') or ((a=='H' or b=='H') and d>=1.5):
            to_drop.append(k)
            continue
        # covalent cutoff
        rsum = COVALENT_RADII.get(a,1.0) + COVALENT_RADII.get(b,1.0)
        if d > rsum + tol:
            to_drop.append(k)
    df = df.drop(to_drop).reset_index(drop=True)
    df[['i','j']] += 1  # to 1-based
    return df[['i','j']]

# Part 1: Read/write XYZ files
def read_xyz(path):
    """
    Reads an XYZ file.
    Returns:
      atoms: list of str, length N
      coords: (N,3)-array of floats
    """
    with open(path) as f:
        natoms = int(f.readline().strip())
        comment = f.readline()  # skip
        atoms, coords = [], []
        for line in f:
            parts = line.split()
            if len(parts) < 4:
                continue
            atoms.append(parts[0])
            coords.append([float(x) for x in parts[1:4]])
    coords = np.array(coords)
    if coords.shape[0] != natoms:
        raise ValueError(f"Expected {natoms} atoms but found {coords.shape[0]}")
    return atoms, coords



# Part 2: Select anchor atom
def select_anchor(atoms, coords, index=None, symbol=None, occurrence=1):
    """
    Selects an anchor atom by index or symbol+occurrence.
    """
    N = len(atoms)
    if index is not None:
        if not (0 <= index < N):
            raise IndexError(f"Index {index} is out of range")
        anchor_idx = index
    elif symbol is not None:
        matches = [i for i, s in enumerate(atoms) if s == symbol]
        if len(matches) < occurrence or occurrence < 1:
            raise ValueError(f"Found {len(matches)} '{symbol}' atoms; cannot select occurrence {occurrence}")
        anchor_idx = matches[occurrence - 1]
    else:
        raise ValueError("Provide either index or symbol+occurrence")
    return anchor_idx, atoms[anchor_idx], coords[anchor_idx]

VDW_RADII = {
    'H': 1.20, 'C': 1.70, 'N': 1.55, 'O': 1.52,
    'F': 1.47, 'P': 1.80, 'S': 1.80, 'Cl':1.75,
    'Br':1.85, 'I':1.98
}

CPK_COLORS = {
    'H': (1.0, 1.0, 1.0),
    'C': (0.2, 0.2, 0.2),
    'N': (0.0, 0.0, 1.0),
    'O': (1.0, 0.0, 0.0),
    'F': (0.0, 0.8, 0.0),
    'P': (1.0, 0.5, 0.0),
    'S': (1.0, 1.0, 0.0),
    'Cl': (0.0, 1.0, 0.0),
    'Br': (0.6, 0.2, 0.2),
    'I':  (0.6, 0.0, 0.6)
}

def get_vdw_radii(atom_list):
    """Return array of VDW radii for host or fragment atoms."""
    return np.array([VDW_RADII.get(a, 1.5) for a in atom_list])

def get_cpk_colors(atom_list):
    """Return list of RGB tuples for each atom (for visualization)."""
    return [CPK_COLORS.get(a, (0.5, 0.5, 0.5)) for a in atom_list]

def score_direction_soft(fragment_coords, fragment_atoms,
                         host_coords, host_vdw_radii,
                         buffer=0.2, exponent=2):
    """
    Vectorized soft clash penalty using VDW radii:
    penalty = sum( max(r_i + r_j + buffer - d_ij, 0) ** exponent )
    Returns: -penalty (higher is better)
    """
    # fragment VDW radii (M,1), host VDW (1,N)
    frag_vdw = get_vdw_radii(fragment_atoms)[:, None]
    # pairwise distances (M,N)
    dists = cdist(fragment_coords, host_coords)
    allowed = frag_vdw + host_vdw_radii[None, :] + buffer
    overlap = np.clip(allowed - dists, 0, None)
    penalty = np.sum(overlap ** exponent)
    return -penalty


def get_radii_array(atoms):
    return np.array([COVALENT_RADII[a] for a in atoms])

def sample_unit_sphere(n_samples):
    i = np.arange(0, n_samples, dtype=float) + 0.5
    phi = np.arccos(1 - 2*i/n_samples)
    theta = np.pi * (1 + np.sqrt(5)) * i
    x = np.cos(theta) * np.sin(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(phi)
    return np.vstack((x, y, z)).T

def rotation_matrix_from_vectors(vec1, vec2):
    a = vec1 / np.linalg.norm(vec1)
    b = vec2 / np.linalg.norm(vec2)
    v = np.cross(a, b)
    c = np.dot(a, b)
    if np.allclose(v, 0):
        return np.eye(3) if c > 0 else -np.eye(3)
    k = np.array([[0,-v[2],v[1]],[v[2],0,-v[0]],[-v[1],v[0],0]])
    return np.eye(3) + k + (k @ k) * (1/(1+c))

# # Updated score_direction with soft penalty scoring
# def score_direction_soft(fragment_coords, fragment_atoms, host_coords, host_radii, buffer=0.2):
#     penalty = 0.0
#     for fa_symbol, fa_coord in zip(fragment_atoms, fragment_coords):
#         r_f = COVALENT_RADII.get(fa_symbol, 1.0)
#         dists = np.linalg.norm(host_coords - fa_coord, axis=1)
#         r_sum = r_f + host_radii + buffer
#         overlap = np.clip(r_sum - dists, 0, None)
#         penalty += np.sum(overlap**2)
#     return -penalty  # more negative = worse

import os



def attach_fragment(host_atoms, host_coords, anchor_idx, fragment_smiles, n_directions=1000, bond_length=1.2, buffer=0.2):
    from rdkit import Chem
    from rdkit.Chem import AllChem

    def build_rdkit_mol_with_coords(atom_list, coords):
        mol = Chem.RWMol()
        for a in atom_list:
            mol.AddAtom(Chem.Atom(a))
        mol = mol.GetMol()
        conf = Chem.Conformer(len(atom_list))
        for i, pos in enumerate(coords):
            conf.SetAtomPosition(i, pos)
        mol.AddConformer(conf)
        Chem.SanitizeMol(mol, sanitizeOps=Chem.SANITIZE_ALL ^ Chem.SANITIZE_PROPERTIES)
        return mol

    def get_fragment_coords_and_bonds(smiles):
        mol = Chem.MolFromSmiles(smiles)
        # mol = Chem.AddHs(mol)
        idx_anchor = [a.GetIdx() for a in mol.GetAtoms() if a.GetSymbol() == '*']
        if len(idx_anchor) != 1:
            raise ValueError("Fragment must contain exactly one wildcard '*' atom.")
        anchor_idx = idx_anchor[0]
        neighbor_idx = [a.GetIdx() for a in mol.GetAtomWithIdx(anchor_idx).GetNeighbors()][0]

        editable = Chem.EditableMol(mol)
        editable.RemoveAtom(anchor_idx)
        mol = editable.GetMol()

        AllChem.EmbedMolecule(mol, AllChem.ETKDG())
        conf = mol.GetConformer()
        coords = np.array([list(conf.GetAtomPosition(i)) for i in range(mol.GetNumAtoms())])
        atoms = [a.GetSymbol() for a in mol.GetAtoms()]

        # Re-index bonds after anchor removed
        bonds = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            order = int(bond.GetBondTypeAsDouble())
            bonds.append((i, j, order))

        return atoms, coords, neighbor_idx, bonds

    def transform_fragment(fragment_coords, fragment_anchor_idx, host_anchor_coord, direction, bond_length=1.5):
        translated = fragment_coords - fragment_coords[fragment_anchor_idx]
        default_dir = np.array([0.0, 0.0, -1.0])
        R = rotation_matrix_from_vectors(default_dir, direction)
        rotated = translated @ R.T
        placed = rotated + (host_anchor_coord + bond_length * direction)
        return placed

    host_anchor = host_coords[anchor_idx]
    host_radii = get_radii_array(host_atoms)

    frag_atoms, frag_coords, frag_anchor_idx, frag_bonds = get_fragment_coords_and_bonds(fragment_smiles)
    dirs = sample_unit_sphere(n_directions)

    best_score = -np.inf
    best_coords = None
    for d in dirs:
        trial_coords = transform_fragment(frag_coords, frag_anchor_idx, host_anchor, d, bond_length)
        score = score_direction_soft(trial_coords, frag_atoms, host_coords, host_radii, buffer)
        if score > best_score:
            best_score = score
            best_coords = trial_coords
    # get backbone  bonds from mol from host_atoms
    host_mol = build_rdkit_mol_with_coords(host_atoms, host_coords)
    host_bonds = []
    for bond in host_mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        order = int(bond.GetBondTypeAsDouble())
        host_bonds.append((i, j, order))

    # Build new atom and coordinate list
    new_atoms = host_atoms + frag_atoms
    new_coords = np.vstack([host_coords, best_coords])

    # Bond indices: host (0..n-1), frag (n..)
    host_N = len(host_atoms)
    adjusted_frag_bonds = [(i + host_N, j + host_N, order) for (i, j, order) in frag_bonds]
    connector_bond = [(anchor_idx, host_N + frag_anchor_idx, 1)]

    all_bonds = adjusted_frag_bonds  + connector_bond  # (no host bonds assumed)

    return new_atoms, new_coords , all_bonds


from openbabel import openbabel, pybel
import numpy as np

def optimize_with_openbabel(atom_list, coords, ffname="UFF", steps=500, bonds=None):
    """
    Build OBMol, add optional bonds, optimize with Open Babel force field.
    
    Parameters:
        atom_list: list of atomic symbols
        coords: (N, 3) numpy array
        ffname: 'UFF', 'MMFF94', etc.
        steps: number of minimization steps
        bonds: list of tuples (i, j, order) with 0-based indices
    """
    from openbabel import openbabel, pybel
    import numpy as np

    obmol = openbabel.OBMol()
    coords = np.asarray(coords)
    original_coords = coords.copy()

    for atom, pos in zip(atom_list, coords):
        obatom = obmol.NewAtom()
        obatom.SetAtomicNum(pybel.ob.GetAtomicNum(atom))
        obatom.SetVector(*pos)

    obmol.ConnectTheDots()           # build a distance‐based graph
    obmol.PerceiveBondOrders()       # assign single/double/triple
    builder = openbabel.OBBuilder()  
    builder.Build(obmol)  
               
    if bonds:
        for i, j, order in bonds:
            obmol.AddBond(i + 1, j + 1, order)  # OBMol is 1-based indexing
            
    obmol.AddHydrogens() 
    # Setup force field
    ff = openbabel.OBForceField.FindForceField(ffname)
    if not ff or not ff.Setup(obmol):
        try:
            ff= openbabel.OBForceField('UFF')
            print(f"Using  force field: 'UFF'")
        except Exception as e:
            print(f"Error setting up force field {ffname}: {e}")
        raise ValueError(f"Could not set up force field {ffname}.")

    ff.SteepestDescent(steps)
    ff.GetCoordinates(obmol)

    coords_opt = np.array([
        [obmol.GetAtom(i).GetX(), obmol.GetAtom(i).GetY(), obmol.GetAtom(i).GetZ()]
        for i in range(1, obmol.NumAtoms() + 1)
    ])

    # RMSD calculation
    if coords.shape == coords_opt.shape:
        rmsd = np.sqrt(np.mean(np.sum((original_coords - coords_opt) ** 2, axis=1)))
        print(f"RMSD after Open Babel optimization: {rmsd:.4f} Å")
    else:
        print("⚠️ Could not compute RMSD (shape mismatch)")

    return coords_opt, obmol

def write_xyz_file(filename, atom_list, coords, comment="optimized by Open Babel"):
    """
    Write atoms and coordinates to an XYZ file.
    """
    with open(filename, 'w') as f:
        f.write(f"{len(atom_list)}\n")
        f.write(f"{comment}\n")
        for atom, (x, y, z) in zip(atom_list, coords):
            f.write(f"{atom} {x:.6f} {y:.6f} {z:.6f}\n")



In [7]:
def build_obmol_from_coords(atom_list, coords):
    """
    Create an OBMol from atom symbols and coords, perceive full chemistry
    using Open Babel (bond orders, aromaticity).
    """
    obmol = openbabel.OBMol()
    for sym, pos in zip(atom_list, coords):
        a = obmol.NewAtom()
        a.SetAtomicNum(pybel.ob.GetAtomicNum(sym))
        a.SetVector(*pos)
    obmol.ConnectTheDots()
    obmol.PerceiveBondOrders()
    builder = openbabel.OBBuilder()
    builder.Build(obmol)
    return obmol

def list_obmol_bonds(obmol):
    """
    Return list of bonds (0-based) in OBMol: (i, j, order, aromatic)
    """
    bonds = []
    for b in openbabel.OBMolBondIter(obmol):
        i0 = b.GetBeginAtomIdx() - 1
        j0 = b.GetEndAtomIdx()   - 1
        bonds.append((i0, j0, b.GetBondOrder(), b.IsAromatic()))
    return bonds

def attach_fragment_debug(host_atoms, host_coords, anchor_idx, fragment_smiles, 
                          n_directions=1000, bond_length=1.2, buffer=0.2):
    """
    Attach fragment like before, but debug bond connectivity at start and end
    using both RDKit and Open Babel perceptions.
    Returns new_atoms, new_coords, all_bonds, debug_info
    """
    # 1. Compute host bonds via RDKit
    def build_rdkit_mol_with_coords(atom_list, coords):
        mol = Chem.RWMol()
        for a in atom_list:
            mol.AddAtom(Chem.Atom(a))
        mol = mol.GetMol()
        conf = Chem.Conformer(len(atom_list))
        for i, pos in enumerate(coords):
            conf.SetAtomPosition(i, pos)
        mol.AddConformer(conf)
        Chem.SanitizeMol(mol, sanitizeOps=Chem.SANITIZE_ALL ^ Chem.SANITIZE_PROPERTIES)
        return mol

    def transform_fragment(fragment_coords, fragment_anchor_idx, host_anchor_coord, direction, bond_length=1.5):
        translated = fragment_coords - fragment_coords[fragment_anchor_idx]
        default_dir = np.array([0.0, 0.0, -1.0])
        R = rotation_matrix_from_vectors(default_dir, direction)
        rotated = translated @ R.T
        placed = rotated + (host_anchor_coord + bond_length * direction)
        return placed
        
    host_rdkit = build_rdkit_mol_with_coords(host_atoms, host_coords)
    host_rdkit_bonds = [(b.GetBeginAtomIdx(), b.GetEndAtomIdx(), int(b.GetBondTypeAsDouble()))
                        for b in host_rdkit.GetBonds()]

    # 2. Compute host bonds via Open Babel
    obmol_host = build_obmol_from_coords(host_atoms, host_coords)
    host_obabel_bonds = list_obmol_bonds(obmol_host)
    host_df = pd.DataFrame({'atom': host_atoms,
                            'x': host_coords[:,0],
                            'y': host_coords[:,1],
                            'z': host_coords[:,2]})
    host_conn = extract_connectivity(host_df)

   
    def get_fragment_coords_and_bonds(smiles):
        mol = Chem.MolFromSmiles(smiles)
        # mol = Chem.AddHs(mol)
        idx_anchor = [a.GetIdx() for a in mol.GetAtoms() if a.GetSymbol() == '*']
        if len(idx_anchor) != 1:
            raise ValueError("Fragment must contain exactly one wildcard '*' atom.")
        anchor_idx = idx_anchor[0]
        neighbor_idx = [a.GetIdx() for a in mol.GetAtomWithIdx(anchor_idx).GetNeighbors()][0]

        editable = Chem.EditableMol(mol)
        editable.RemoveAtom(anchor_idx)
        mol = editable.GetMol()

        AllChem.EmbedMolecule(mol, AllChem.ETKDG())
        conf = mol.GetConformer()
        coords = np.array([list(conf.GetAtomPosition(i)) for i in range(mol.GetNumAtoms())])
        atoms = [a.GetSymbol() for a in mol.GetAtoms()]

        # Re-index bonds after anchor removed
        bonds = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            order = int(bond.GetBondTypeAsDouble())
            bonds.append((i, j, order))

        return atoms, coords, neighbor_idx, bonds

    frag_atoms, frag_coords, frag_anchor_idx, frag_bonds = get_fragment_coords_and_bonds(fragment_smiles)

    # 4. Choose best placement
    host_anchor = host_coords[anchor_idx]
    host_radii = get_radii_array(host_atoms)
    dirs = sample_unit_sphere(n_directions)
    best_score, best_coords = -np.inf, None
    for d in dirs:
        trial = transform_fragment(frag_coords, frag_anchor_idx, host_anchor, d, bond_length)
        sc = score_direction_soft(trial, frag_atoms, host_coords, host_radii, buffer)
        if sc > best_score:
            best_score, best_coords = sc, trial

    # 5. Build new atom list and coords
    new_atoms = host_atoms + frag_atoms
    new_coords = np.vstack([host_coords, best_coords])
    host_N = len(host_atoms)
    frag_adj_bonds = [(i+host_N, j+host_N, order) for (i, j, order) in frag_bonds]
    connector = [(anchor_idx, host_N+frag_anchor_idx, 1)]
    all_bonds = host_rdkit_bonds + frag_adj_bonds + connector
    new_df = pd.DataFrame({'atom': new_atoms,
                           'x': new_coords[:,0],
                           'y': new_coords[:,1],
                           'z': new_coords[:,2]})
    new_conn = extract_connectivity(new_df)

    # 6. Compute final OBMol bonds
    obmol_final = build_obmol_from_coords(new_atoms, new_coords)
    final_obabel_bonds = list_obmol_bonds(obmol_final)
    # Compare connectivity between the host  and new conn
    try:
        comoare_conn = pd.merge(host_conn, new_conn, on=['i', 'j'], how='outer', indicator=True)
        if comoare_conn['_merge'].value_counts().get('both', 0) == len(host_conn):
            print("All original host bonds are still present in the final structure.")
        else:
            print("Some host bonds are missing in the final structure.")
    except Exception as e:
        print(f"Error comparing connectivity: {e}")
        
    debug_info = {
        'host_rdkit_bonds': host_rdkit_bonds,
        'host_obabel_bonds': host_obabel_bonds,
        'fragment_bonds': frag_bonds,
        'initial_all_bonds': all_bonds,
        'final_obabel_bonds': final_obabel_bonds,
        'best_score': best_score
    }

    return new_atoms, new_coords, all_bonds, debug_info

def report_bond_changes(debug_info):
    """
    debug_info must contain:
      - host_rdkit_bonds:    list of (i,j,order)
      - fragment_bonds:      list of (i,j,order) in frag coords
      - initial_all_bonds:   list of (i,j,order) merged before FF
      - final_obabel_bonds:  list of (i,j,order,aromatic) after FF
    """
    hr  = debug_info['host_rdkit_bonds']
    fb  = debug_info['fragment_bonds']
    init= debug_info['initial_all_bonds']
    fb_only = [(i,j,o) for (i,j,o) in init if (i,j,o) not in hr]
    conn = [b for b in fb_only if b not in fb]
    
    # Normalize final bonds to (i,j,order)
    fin = [(i,j,o) for (i,j,o,ar) in debug_info['final_obabel_bonds']]

    print("\n=== ORIGINAL HOST BONDS (RDKit) ===")
    for i,j,o in sorted(hr):
        print(f"  {i}–{j} order={o}")

    print("\n=== FRAGMENT INTERNAL BONDS ===")
    for i,j,o in sorted(fb):
        print(f"  frag {i}–{j} order={o}")

    print("\n=== CONNECTOR BOND ===")
    for i,j,o in conn:
        print(f"  {i}–{j} order={o}")

    print("\n=== ALL MERGED BONDS (before FF) ===")
    for i,j,o in sorted(init):
        tag = ("host" if (i,j,o) in hr else
               "frag" if (i,j,o) in fb else
               "conn")
        print(f"  {i}–{j} order={o}  [{tag}]")

    print("\n=== FINAL PERCEIVED BONDS (OpenBabel) ===")
    for i,j,o in sorted(fin):
        arom = next(ar for (ii,jj,oar,ar) in debug_info['final_obabel_bonds']
                    if (ii,jj,oar)==(i,j,o))
        print(f"  {i}–{j} order={o} aromatic={arom}")

    # check that all host bonds survived
    print("\n=== HOST BONDS MISSING IN FINAL? ===")
    missing = [b for b in hr if (b[0],b[1],b[2]) not in fin and (b[1],b[0],b[2]) not in fin]
    if not missing:
        print("  All original host bonds are still present.")
    else:
        for i,j,o in missing:
            print(f"  MISSING: {i}–{j} order={o}")



In [9]:
import os
from glob import glob
from scipy.spatial.distance import cdist
def process_single_xyz(input_path, fragment_smiles,
                       anchor_index=None, anchor_symbol=None, anchor_occurrence=1,
                       n_directions=1000, bond_length=1.1, buffer=0.2,
                       ffname='MMFF94', steps=1000):
    """
    Process one XYZ file: attach fragment and optimize.
    """
    # Read input
    atoms, coords = read_xyz(input_path)
    df= pd.DataFrame({'atom': atoms, 'x': coords[:,0], 'y': coords[:,1], 'z': coords[:,2]})
    show_single_molecule('before',df)
    # Select anchor (by index or symbol+occurrence)
    idx, sym, coord = select_anchor(
        atoms, coords,
        index=anchor_index,
        symbol=anchor_symbol,
        occurrence=anchor_occurrence
    )
    # Attach fragment
    new_atoms, new_coords, bonds , debug_info = attach_fragment_debug(
        atoms, coords, idx,
        fragment_smiles=fragment_smiles,
        n_directions=n_directions,
        bond_length=bond_length,
        buffer=buffer
    )
    # report_bond_changes(debug_info)
    # Optimize geometry
    coords_opt, obmol = optimize_with_openbabel(
        new_atoms, new_coords,
        ffname=ffname, steps=steps,
        bonds=bonds
    )
    new_df = pd.DataFrame({
        'atom': new_atoms,
        'x': coords_opt[:, 0],
        'y': coords_opt[:, 1],
        'z': coords_opt[:, 2]
    })
    show_single_molecule('after', new_df)
    # Write result
    base, _ = os.path.splitext(os.path.basename(input_path))
    output_path = os.path.join(os.path.dirname(input_path),
                               f"{base}_modified.xyz")
    write_xyz_file(output_path, new_atoms, coords_opt,
                   comment=f"Fragment {fragment_smiles} on {sym}{idx}")
    print(f"Processed {input_path} → {output_path}")
    return output_path

def process_directory(directory, fragment_smiles,
                      anchor_index=None, anchor_symbol=None, anchor_occurrence=1,
                      n_directions=1000, bond_length=1.2, buffer=0.5,
                      ffname='MMFF94', steps=500):
    """
    Process all .xyz files in a directory.
    """
    # use glob to find all .xyz files without modified suffix '_modified' 
    xyz_files = glob(os.path.join(directory, '*.xyz'))
    xyz_files = [f for f in xyz_files if not f.endswith('_modified.xyz')]
    for xyz in xyz_files:
        process_single_xyz(
            xyz, fragment_smiles,
            anchor_index, anchor_symbol, anchor_occurrence,
            n_directions, bond_length, buffer,
            ffname, steps
        )

# Example usage:
# process_single_xyz('L19.xyz', '[*]CC', anchor_index=1)
process_directory(r'C:\Users\edens\Documents\GitHub\MolAlign\MolAlign\xyz_files\hidaya', '[*]C', anchor_index=1, anchor_symbol='P', anchor_occurrence=1,ffname='UFF')

# If you want a CLI, you can wrap these in argparse as needed.

🔥 sterimol_trace failed: TypeError("'NoneType' object is not subscriptable")


[10:51:16] Molecule does not have explicit Hs. Consider calling AddHs()


IndexError: index 1 is out of bounds for axis 0 with size 1

In [4]:
os.chdir(r'C:\Users\edens\Documents\GitHub\LabCode\MolFeatures\utils')
from visualize import show_single_molecule